# Kanapsack Problem databases 06:

* https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html

# Atividade com nota:

* Avaliar o algoritmo Hill Climbing para as bases P01 a P07;
* Utilizar a função de aptidão knapsack do Mlrose;
* Apresentar a melhor solução encontrada e comparar com a melhor solução global disponível para a base de dados



In [1]:
!pip install mlrose

In [2]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np

## Problem representation:
### 0/1 knapsack problem using a hill climbing algorithm for P06.
- The Problem: 
Given a set of items, each with a weight and a value, select a subset of the items to maximize the total value while keeping the total weight within a given capacity.

So the total_weight of any system configuration (any state) is constraint with a max_weight, I will call this max_weight W.


In [3]:
max_weight = 170
weights = [41,50,49,59,55,57,60]
values = [442, 525, 511, 593, 546, 564,617]
state = np.array([0, 1, 0, 1, 0, 0, 1])
len(weights)

7

In [4]:
items = []
for i in range(len(weights)):
    items.append(('Item' + str(i), weights[i], values[i]))

items

[('Item0', 41, 442),
 ('Item1', 50, 525),
 ('Item2', 49, 511),
 ('Item3', 59, 593),
 ('Item4', 55, 546),
 ('Item5', 57, 564),
 ('Item6', 60, 617)]

This get_cost function is what I want to maximize. It's used to see which neighbor is better.

In [5]:
# Knapsack fitness function
def get_cost(state):
    total_value = sum(state[i] * values[i] for i in range(len(weights)))
    total_weight = sum(state[i] * weights[i] for i in range(len(weights)))
    if total_weight > max_weight:
        return 0
    return total_value

Which is nothing but this in an equation form:

$F(x_i) = \sum_{i=0}^{n-1} x_i v_i$,    if $\sum_{i=0}^{n-1} x_i w_i \leq W$ this is the constraint


Where $x_i$ represents a state vector $x = [x_0, x_1, \ldots, x_{n-1}]$. It denotes a number of copies of item i included in the knapsack.

In [6]:
# evaluate fitness function of given state
total_value = get_cost(state)
print("Fitness value: ", total_value)

Fitness value:  1735


In [7]:
# custom fitness function object
fitness_cust = mlrose.CustomFitness(get_cost)

In [8]:
#create enviroment
problem = mlrose.DiscreteOpt(length = 7, fitness_fn = fitness_cust, maximize = True, max_val = 2)

### Hill Climb Method
- Main Idea: 
The algorithm iteratively improves the solution by exploring neighboring states and selecting the state with the highest value (accordingly with get_cost) until no better solution can be found or a maximum number of iterations is reached (100 below).
In other words, this algorithm maintain a single node and searches by moving to a neighboring node.

In [9]:
# Call the hill_climb function to solve the problem
best_state, best_fitness, curve = mlrose.hill_climb(problem, max_iters=100, restarts=0, init_state = state, curve=True)
best_state, best_fitness, curve

(array([0, 1, 0, 1, 0, 0, 1]), 1735.0, array([], dtype=float64))

In [10]:
get_cost(best_state)

1735

In [11]:
print(f"soluçãoHC: {best_state}")
print(f"Fitness Value: {best_fitness}")

soluçãoHC: [0 1 0 1 0 0 1]
Fitness Value: 1735.0


### Introducing Random Restart 
Randomness should improve the optimization process value. Because for each iteration, it explores neighboring states by flipping the value (0 to 1 or 1 to 0) of a randomly selected item, and evaluates their costs and it keeps track of the best neighboring states with the highest cost.The current state is then updated to one of the best neighboring states. The process continues until no better neighbor is found or the maximum number of iterations is reached.

In [12]:
best_state, best_fitness, curve = mlrose.random_hill_climb(problem, max_iters=100, restarts=10, curve=True, random_state=10)
best_state, best_fitness, curve

(array([0, 1, 0, 1, 0, 0, 1]),
 1735.0,
 array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0., 1139., 1139., 1581., 1581., 1581., 1581., 1581., 1581.,
        1581., 1581., 1581., 1581., 1581.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0., 1582., 1582., 1582.,
        1582., 1582., 1582., 1582., 1582., 1582., 1582., 1599., 1599.,
        1599., 1599., 1599., 1599., 1599., 1599., 1599., 1599., 1599.,
        1668., 1668., 1668., 1668., 1668., 1668., 1668., 1668., 1668.,
        1668., 1668., 1629., 1629., 1629., 1629., 1629., 1629., 1629.,
        1629., 1629., 1629., 1650., 1650., 1650., 1650., 1650., 1650.,
        1650., 1650., 1650., 1650., 1621., 1621., 1621., 1621., 1621.,
        1621., 1621., 1621., 1621., 1621., 1478., 1478., 1478., 1478.,
        1478., 1478., 1478., 1478., 1478., 1478., 1735., 1735., 1735.,
        1735., 1735., 1735., 1735., 1735., 1735., 1735., 1735.]))

## Conclusion
Rather than just hill climb one time, we can hill climb multiple times and figure out what is the best one that I've been able to find.
So we've implemented a function for random restart that restarts some maximum number of times, and repeat, on that number of times, this hill-climb method 'random_hill_climb' and figure out what the cost is (for that state after randomly restart maximum times).
As we can see there is a lot of local maximums (1139, 1581, 1582...),  the algorithm starts at 1139 and runs through a lot of neighbors, always keeping track (even the lowest ones) of the best neighboring states with the highest cost. 1735 is the global maximum for this dataset. So the given optimal solution was already a global maximum.

# Simulated Annealing
In order to find a global maximum or global minimum we may need to make a move that makes our situation worse. Because sometimes if we get stuck in a local max/min, I want to dislodge from that in order to find the global max/min.
 - Simulated Annealing is a great technique for that. 
Suppose some state-space, if I am in a current state and I'm looking for a global maximum and I'm trying to maximize the value of the state, Hill-Climbing would just take the state and look at the two neighbor ones, and always pick the one that is going to increase the value of the state. As said before "in order to find a global maximum/minimum we may need to make a move that makes our situation worse." such that later on we can find that global maximum.
Once found this global max state we probably don't wan't to be moving to states worse than current state. This is why this metaphor for annealing -> start making more random moves and, over time, start to make fewer of those random moves based on a particular temperature schedule.

In [13]:
# Define a schedule for simulated annealing (you can customize this schedule)
schedule = mlrose.ExpDecay(init_temp=1000, exp_const=0.01, min_temp=1)

the idea is that this temperature is going to be higher early on and lower later on.
It is exponentially decaying according to the formula: $T(t) = T_{i}e^{-rt}$
where:
-  T(t) is the temperature at time t.
- $T(i)$ is the initial temperature at time t=0.
- r is the rate of exponential decay.
- t is the time variable.
- The exponential decay is represented by $e^{-rt}$.

For ex: You start off $T_{i}$=100, after 1s the temperature will be:

In [14]:
# Evaluate the temperature parameter at time t = 1 and exp_const=0.01
t1 = schedule.evaluate(1)
print(f"T(1) = {t1}")

T(1) = 990.0498337491681


In [15]:
# Call the simulated_annealing function to solve the problem
best_state, best_fitness, curve = mlrose.simulated_annealing(problem, schedule=schedule, max_attempts=50, init_state=None, curve=True, random_state=10)
best_state, best_fitness, curve

(array([0, 1, 0, 1, 0, 0, 1]),
 1735.0,
 array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0., 1650., 1057., 1621., 1621., 1621., 1621., 1075.,
        1075., 1692., 1692., 1181., 1692., 1692., 1692., 1075., 1621.,
        1621., 1621., 1621., 1621., 1621., 1621., 1621.,    0.,    0.,
           0.,    0.,    0., 1706., 1706., 1181., 1692., 1128., 1653.,
        1653., 1653., 1653., 1036., 1629., 1104., 1629., 1629., 1104.,
        1629., 1629., 1629., 1629., 1629., 1629., 1629., 1629., 1118.,
        1118., 1735., 1118., 1118., 1682., 1682., 1089., 1682., 1682.,
        1682., 1682., 1682., 1682., 1682., 1682., 1157., 1682., 1118.,
         593., 1210., 1210., 1735., 1735., 1210., 1210., 1210., 1721.,
        1721., 1721., 1721., 1721., 1721., 1721., 1721., 1104., 1721.,
        1104., 1546., 1546., 1546., 1546., 1546., 1546., 1546., 1546.,
        1546., 1546., 1035., 1035., 1546., 1546., 1546., 1546., 1546.,
         953., 1517., 1517., 1517., 1

# Conclusion
so the goal of this whole process is as we begin our search to find the local/global max/min, we can dislodge ourselves if we get stuck at a local max/min in order to eventually make our way to exploring the part of the state space that is going to be the best. And then as the temperature decreases, start to make fewer of those random moves, that is, not moving aroung too much to get into another part of the state space.


The simulated annealing algorithm ran through numerous iterations to find an optimal combination of items to include in a knapsack. The curve data indicates how the solution evolved over iterations, with the algorithm converging on the optimal solution towards the end.

**Important**: In order for finding the global maximum, I had to **adjust the parameters** in the ExpDecay schedule and in the simulated_annealing function.

- **Best State**: [0, 1, 0, 1, 0, 0, 1] This is an array representing the optimal solution found by the algorithm.
- **Best fitness**: 1735 says that the combination of items in the best state yields a total value of 1735.
- **Curve**: This is an array showing the fitness score at each iteration of the algorithm. It helps in understanding how the solution improved over time. The values fluctuate, indicating the exploration of different solutions. The repeated values of 1735 towards the end suggest that the algorithm consistently found a solution with a fitness of 1735, indicating it likely reached an optimal or near-optimal solution